<a href="https://colab.research.google.com/github/armin1380/gan_stegano_audio/blob/main/steganography_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/audiou_stegano
!git clone https://github.com/Chenlang2018/Audio-Steganography-using-GAN.git
from google.colab import files
files.upload()  # Select your kaggle.json file here
!pip install kaggle
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mfekadu/darpa-timit-acousticphonetic-continuous-speech
!mkdir /content/drive/MyDrive/TIMIT/audiou_stegano/TIMIT
!unzip darpa-timit-acousticphonetic-continuous-speech.zip -d /content/drive/MyDrive/TIMIT/audiou_stegano/TIMIT


/content/drive/MyDrive/audiou_stegano
Cloning into 'Audio-Steganography-using-GAN'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 62 (delta 21), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (62/62), 22.76 KiB | 2.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/mfekadu/darpa-timit-acousticphonetic-continuous-speech
License(s): copyright-authors
100% 825M/829M [00:13<00:00, 93.9MB/s]
100% 829M/829M [00:13<00:00, 62.8MB/s]
mkdir: cannot create directory ‘/content/drive/MyDrive/TIMIT/audiou_stegano/TIMIT’: No such file or directory
Archive:  darpa-timit-acousticphonetic-continuous-speech.zip
checkdir:  cannot create extraction directory: /content/drive/MyDrive/TIMIT/audiou_stegano/TIMIT
           No such file or directory


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/audiou_stegano
!unzip darpa-timit-acousticphonetic-continuous-speech.zip -d /content/drive/MyDrive/audiou_stegano/TIMIT

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA1.WAV  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA1.WAV.wav  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA1.WRD  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA2.PHN  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA2.TXT  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA2.WAV  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA2.WAV.wav  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SA2.WRD  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SI1060.PHN  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN/DR6/MTXS0/SI1060.TXT  
  inflating: /content/drive/MyDrive/audiou_stegano/TIMI

In [ ]:
import os
import shutil
import random

# Path to the dataset and target directories
dataset_path = '/content/drive/MyDrive/audiou_stegano/TIMIT/data/TRAIN'
target_dataset_path = '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/small_dataset'

# Create a new directory for the small dataset
os.makedirs(target_dataset_path, exist_ok=True)

# Recursively find all .wav files in the dataset
all_audio_files = []
for root, _, files in os.walk(dataset_path):
    all_audio_files.extend([os.path.join(root, f) for f in files if f.endswith('.wav')])

# Check the total number of available files
print(f"Total .wav files found: {len(all_audio_files)}")

# Adjust the sample size if there are fewer than 1,000 files
sample_size = min(len(all_audio_files), 1000)

# Randomly select files
sampled_files = random.sample(all_audio_files, sample_size)

# Copy selected files to the target directory
for file in sampled_files:
    shutil.copy(file, target_dataset_path)

print(f"Copied {len(sampled_files)} files to {target_dataset_path}")

Total .wav files found: 3741
Copied 1000 files to /content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/small_dataset


In [ ]:
!pip install pydub
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
from pydub import AudioSegment
import os

# Path to your 1000-item dataset
dataset_path = '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/small_dataset'

# Process each file in the dataset
for file in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file)

    if file.endswith('.wav'):
        # Load audio file
        audio = AudioSegment.from_file(file_path)

        # Resample to 16 kHz and mono
        audio = audio.set_frame_rate(16000).set_channels(1)

        # Trim or pad to 16,384 samples (~1.024 seconds at 16 kHz)
        duration_ms = len(audio)  # Duration in milliseconds
        target_duration_ms = int((16384 / 16000) * 1000)  # Convert to milliseconds
        if duration_ms > target_duration_ms:
            audio = audio[:target_duration_ms]  # Trim
        else:
            padding = target_duration_ms - duration_ms
            silence = AudioSegment.silent(duration=padding)  # Silence padding
            audio = audio + silence

        # Save the processed audio (overwrite the original)
        audio.export(file_path, format="wav")

print("All files have been resampled to 16 kHz mono and adjusted to 16,384 samples.")


All files have been resampled to 16 kHz mono and adjusted to 16,384 samples.


In [ ]:
import wave
import os

# Path to the dataset
dataset_path = '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/small_dataset'
# File to save the list of problematic files
problematic_files_list = '/content/drive/MyDrive/audiou_stegano/problematic_files.txt'

problematic_files = []

# Validate each file in the dataset
for file in os.listdir(dataset_path):
    file_path = os.path.join(dataset_path, file)
    if file.endswith('.wav'):
        with wave.open(file_path, 'rb') as wav_file:
            sample_rate = wav_file.getframerate()
            channels = wav_file.getnchannels()
            n_frames = wav_file.getnframes()

            if sample_rate != 16000 or channels != 1 or n_frames != 16384:
                problematic_files.append(file_path)

# Save the list to a text file
with open(problematic_files_list, 'w') as f:
    for file in problematic_files:
        f.write(file + '\n')

print(f"Total problematic files: {len(problematic_files)}")
print(f"List saved to {problematic_files_list}")

from pydub import AudioSegment

# Process the problematic files
for file_path in problematic_files:
    # Load the audio file
    audio = AudioSegment.from_file(file_path)

    # Resample to 16 kHz and mono
    audio = audio.set_frame_rate(16000).set_channels(1)

    # Adjust to 16,384 samples (~1.024 seconds at 16 kHz)
    duration_ms = len(audio)
    target_duration_ms = int((16384 / 16000) * 1000)
    if duration_ms > target_duration_ms:
        audio = audio[:target_duration_ms]  # Trim
    else:
        padding = target_duration_ms - duration_ms
        silence = AudioSegment.silent(duration=padding)  # Add silence
        audio = audio + silence

    # Overwrite the file with the corrected audio
    audio.export(file_path, format="wav")

print("All problematic files have been corrected.")



Total problematic files: 0
List saved to /content/drive/MyDrive/audiou_stegano/problematic_files.txt
All problematic files have been corrected.


In [ ]:
import os

def list_directory_structure(directory_path, output_file):
    """
    Recursively generates a tree-like structure of all files and subdirectories under
    the given directory and saves it to the specified output file.

    Args:
        directory_path (str): Path to the directory to scan.
        output_file (str): Path to the file where the structure will be saved.
    """
    def write_tree(dir_path, prefix=""):
        for root, dirs, files in os.walk(dir_path):
            # Write current directory
            f.write(f"{prefix}Directory: {root}\n")
            # Write subdirectories
            for dir_name in dirs:
                f.write(f"{prefix}  Subdirectory: {dir_name}\n")
            # Write files
            for file_name in files:
                f.write(f"{prefix}  File: {file_name}\n")

    with open(output_file, 'w') as f:
        write_tree(directory_path)

    print(f"Tree structure saved to {output_file}")

# Example usage:
directory_path = '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN'
output_file = '/content/drive/MyDrive/audiou_stegano/dataset_structure.txt'
list_directory_structure(directory_path, output_file)




Tree structure saved to /content/drive/MyDrive/audiou_stegano/dataset_structure.txt


In [ ]:
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))


GPU Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN')
sys.path.append('/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer')
print(sys.path)
try:
  import model_sngan
  print("Module imported successfully!")
except ModuleNotFoundError:
  print("Module not found. Please check the path and module name.")

['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN', '/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer']
Module not found. Please check the path and module name.


In [ ]:
import os
os.chdir('/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/snlayer')
import model_sngan

In [ ]:
%cd /content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN/
%ls


/content/drive/MyDrive/audiou_stegano/Audio-Steganography-using-GAN
'Copy of README.md'   max_sv.py        __pycache__/     snconv1d.py
 main_sngan.py        model_sngan.py   small_dataset/   snlinear.py


In [ ]:
!python main_sngan.py

Training started!
Epoch [1/100], Step [5/17], D Loss: 4.918893, G Loss: -0.768619
Epoch [1/100], Step [10/17], D Loss: 4.787293, G Loss: -0.482221
Epoch [1/100], Step [15/17], D Loss: 4.586003, G Loss: -0.298376
Epoch [2/100], Step [5/17], D Loss: 4.069099, G Loss: -0.028988
Epoch [2/100], Step [10/17], D Loss: 4.000809, G Loss: -0.001064
Epoch [2/100], Step [15/17], D Loss: 3.998957, G Loss: -0.000501
Epoch [3/100], Step [5/17], D Loss: 3.998992, G Loss: -0.000192
Epoch [3/100], Step [10/17], D Loss: 3.999240, G Loss: -0.000010
Epoch [3/100], Step [15/17], D Loss: 3.998463, G Loss: -0.000204
Epoch [4/100], Step [5/17], D Loss: 3.998541, G Loss: -0.000003
Epoch [4/100], Step [10/17], D Loss: 3.999170, G Loss: -0.000001
Epoch [4/100], Step [15/17], D Loss: 3.998096, G Loss: -0.000090
Epoch [5/100], Step [5/17], D Loss: 3.998731, G Loss: -0.000026
Epoch [5/100], Step [10/17], D Loss: 3.996650, G Loss: -0.000004
Epoch [5/100], Step [15/17], D Loss: 3.997783, G Loss: -0.000023
Epoch [6/100